# Task 1.1

In [3]:
import pandas as pd
data = {
    "No": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
    "System": ["SK", "SRK", "SK", "SK", "SMS", "SK", "SRK", "SMS", "SRK", "SRK", "SRK", "SRK", "SK", "SMS"],
    "mount": ["F", "F or R", "F", "F", "F or R", "F", "F or R", "F or R", "F or R", "F or R", "F or R", "F", "F", "F or R"],
    "price": [206, 520, 160, 213, 598, 109, 325, 498, 396, 325, 389, 298, 149, 684],
    "con": [1.9, 2.1, 1.7, 1.7, 1.6, 2.0, 2.0, 1.5, 2.8, 2.2, 2.0, 2.5, 1.9, 1.7],
    "snow": [1.4, 0.8, 1.9, 2.0, 2.4, 1.9, 2.1, 3.3, 2.1, 2.2, 2.2, 2.3, 2.5, 3.3],
    "ice": [1.8, 3.8, 1.6, 2.4, 7.0, 2.4, 3.2, 3.5, 3.1, 4.6, 3.3, 3.3, 4.0, 4.4],
    "dur": [2.7, 2.3, 3.7, 3.4, 2.8, 3.7, 2.8, 2.0, 2.5, 3.2, 4.3, 2.8, 3.8, 2.2],
    "Accegrade": ["F", "F", "F", "F", "F", "F", "F", "T", "T", "T", "T", "T", "T", "T"]
}

test_data = {
    "No": [15, 16, 17],
    "System": ["SK", "SK", "SK"],
    "mount": ["F", "F", "F"],
    "price": [99, 140, 215],
    "con": [2.8, 2.6, 2.3],
    "snow": [2.2, 2.3, 3.8],
    "ice": [2.5, 3.3, 4.8],
    "dur": [4.0, 3.4, 2.3],
    "Accegrade": ["T", "T", "T"]
}
df = pd.DataFrame(data)
test_df = pd.DataFrame(test_data)


for col in ["price", "con","snow","ice","dur"]:
    df[col] = df[col].apply(lambda x: (x, x))
    test_df[col] = test_df[col].apply(lambda x: (x, x))

# Преобразование DataFrame в список словарей для классификации
banks = df.to_dict(orient='records')
test_banks = test_df.to_dict(orient='records')

len(banks)

# Функция для вычисления пересечения интервалов
def interval_intersection(interval1, interval2):
    return min(interval1[0], interval2[0]), max(interval1[1], interval2[1])

# Функция для вычисления IntSec
def compute_intsec(bank, test_bank):
    intsec = {}
    for key in bank:
        if key in test_bank and key != 'No' and key != 'Accegrade': # Исключаем ненужные ключи
            if isinstance(bank[key], tuple):  # Для интервальных атрибутов
                intsec[key] = interval_intersection(bank[key], test_bank[key])
            elif bank[key] == test_bank[key]:  # Для номинальных атрибутов
                intsec[key] = bank[key]
    return intsec

# Функция для вычисления Ext
def compute_ext(banks, intsec):
    ext = []
    for index, bank in enumerate(banks):
        match = True
        for key in intsec:
            if key in bank:
                if isinstance(intsec[key], tuple):  # Для интервальных атрибутов
                    if not (intsec[key][0] <= bank[key][0] <= intsec[key][1]):
                        match = False
                        break
                else:  # Для номинальных атрибутов
                    if intsec[key] != bank[key]:
                        match = False
                        break
            else:
                match = False
                break
        if match:
            ext.append(index)
    return ext

# Алгоритм Ленивой Классификации
def lazy_classification(banks, test_bank):
    for bank in banks:
        intsec = compute_intsec(bank, test_bank)
        #print(f"{bank['No']}:IntSec -  {intsec}")
        ext = compute_ext(banks, intsec)
        #print(f"{bank['No']}:Ext - {ext}")

        if len(ext) > 0:  # Проверка, есть ли элементы в Ext
            if all(banks[i]["Accegrade"] == "T" for i in ext):
                return "T"
            elif all(banks[i]["Accegrade"] == "F" for i in ext):
                return "F"
            else:
                print("Undetermined")
        else:
            print("Undetermined")


# Проходим по всем объектам тестового набора и классифицируем их
step = 15
for test_bank in test_banks:
    print(f"For {step} row")
    classification_result = lazy_classification(banks, test_bank)
    print(f"Classification for {test_bank['No']}: {classification_result}")
    step = step + 1

For 15 row
Classification for 15: F
For 16 row
Classification for 16: F
For 17 row
Classification for 17: F


# Task 1.2

In [23]:
data = {
    "No": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
    "System": ["SK", "SRK", "SRK", "SK", "SMS", "SK", "SRK", "SMS", "SRK", "SK", "SRK", "SRK", "SK", "SMS"],
    "mount": ["F", "F or R", "F or R", "F", "F or R", "F", "F or R", "F or R", "F or R", "F", "F or R", "F", "F", "F or R"],
    "price": [149, 520, 389, 213, 598, 109, 325, 498, 396, 160, 389, 298, 206, 684],
    "con": [1.9, 2.1, 2.0, 1.7, 1.6, 2.0, 2.0, 1.5, 2.8, 1.7, 2.0, 2.5, 1.9, 1.7],
    "snow": [2.5, 0.8, 2.2, 2.0, 2.4, 1.9, 2.1, 3.3, 2.1, 1.9, 2.2, 2.3, 1.4, 3.3],
    "ice": [4.0, 3.8, 3.3, 2.4, 7.0, 2.4, 3.2, 3.5, 3.1, 1.6, 3.3, 3.3, 1.8, 4.4],
    "dur": [3.8, 2.3, 4.3, 3.4, 2.8, 3.7, 2.8, 2.0, 2.5, 3.7, 4.3, 2.8, 2.7, 2.2],
    "Accegrade": ["T", "F", "T", "F", "F", "F", "F", "T", "T", "F", "T", "T", "F", "T",]
}

test_data = {
    "No": [15, 16, 17],
    "System": ["SK", "SK", "SK"],
    "mount": ["F", "F", "F"],
    "price": [99, 140, 215],
    "con": [2.8, 2.6, 2.3],
    "snow": [2.2, 2.3, 3.8],
    "ice": [2.5, 3.3, 4.8],
    "dur": [4.0, 3.4, 2.3],
    "Accegrade": ["T", "T", "T"]
}
df = pd.DataFrame(data)
test_df = pd.DataFrame(test_data)


for col in ["price", "con","snow","ice","dur"]:
    df[col] = df[col].apply(lambda x: (x, x))
    test_df[col] = test_df[col].apply(lambda x: (x, x))

# Преобразование DataFrame в список словарей для классификации
banks = df.to_dict(orient='records')
test_banks = test_df.to_dict(orient='records')

len(banks)

# Функция для вычисления пересечения интервалов
def interval_intersection(interval1, interval2):
    return max(interval1[0], interval2[0]), 1000

# Функция для вычисления IntSec
def compute_intsec(bank, test_bank):
    intsec = {}
    for key in bank:
        if key in test_bank and key != 'No' and key != 'Accegrade': # Исключаем ненужные ключи
            if isinstance(bank[key], tuple):  # Для интервальных атрибутов
                intsec[key] = interval_intersection(bank[key], test_bank[key])
            elif bank[key] == test_bank[key]:  # Для номинальных атрибутов
                intsec[key] = bank[key]
    return intsec

# Функция для вычисления Ext
def compute_ext(banks, intsec):
    ext = []
    for index, bank in enumerate(banks):
        match = True
        for key in intsec:
            if key in bank:
                if isinstance(intsec[key], tuple):  # Для интервальных атрибутов
                    if not (intsec[key][0] <= bank[key][0] <= intsec[key][1]):
                        match = False
                        break
                else:  # Для номинальных атрибутов
                    if intsec[key] != bank[key]:
                        match = False
                        break
            else:
                match = False
                break
        if match:
            ext.append(index)
    return ext

# Алгоритм Ленивой Классификации
def lazy_classification(banks, test_bank):
    for bank in banks:
        intsec = compute_intsec(bank, test_bank)
        #print(f"{bank['No']}:IntSec -  {intsec}")
        ext = compute_ext(banks, intsec)
        #print(f"{bank['No']}:Ext - {ext}")

        if len(ext) > 0:  # Проверка, есть ли элементы в Ext
            if all(banks[i]["Accegrade"] == "T" for i in ext):
                return "T"
            elif all(banks[i]["Accegrade"] == "F" for i in ext):
                return "F"
            else:
                print("Undetermined")
        else:
            print("Undetermined")


# Проходим по всем объектам тестового набора и классифицируем их
step = 15
for test_bank in test_banks:
    print(f"For {step} row")
    classification_result = lazy_classification(banks, test_bank)
    print(f"Classification for {test_bank['No']}: {classification_result}")
    step = step + 1

For 15 row
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Classification for 15: None
For 16 row
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Classification for 16: None
For 17 row
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Undetermined
Classification for 17: None


# Task 1.3

In [24]:
from sklearn.model_selection import KFold

# Объединение исходного и тестового наборов данных
combined_data = pd.concat([df, test_df]).reset_index(drop=True)
print(combined_data)
# Подготовка к 5-кратной перекрёстной проверке
kf = KFold(n_splits=5, shuffle=True, random_state=42)
def run_lazy_classification(train, test):
    banks = train.to_dict(orient='records')
    test_banks = test.to_dict(orient='records')

    correct_predictions = 0
    for test_bank in test_banks:
        predicted_label = lazy_classification(banks, test_bank)  # Предсказанное значение
        true_label = test_bank['Accegrade']  # Истинное значение

        if predicted_label == true_label:
            correct_predictions += 1

    accuracy = correct_predictions / len(test_banks)
    return accuracy

# Инициализация списка для хранения результатов точности
accuracies = []

custom_test_indices = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8],
    [9, 10, 11],
    [12, 13, 14]
]

for test_index in custom_test_indices:
    # Индексы для тренировочного набора - все, кроме тестовых
    train_index = [i for i in range(len(combined_data)) if i not in test_index]
    
    train, test = combined_data.iloc[train_index], combined_data.iloc[test_index]
    accuracy = run_lazy_classification(train, test)
    accuracies.append(accuracy)

# Просмотр результатов
for i, test_index in enumerate(custom_test_indices):
    print(f"Fold {i+1}:")
    print(f"Train Indices: {[i for i in range(len(combined_data)) if i not in test_index]}")
    print(f"Test Indices: {test_index}")
    print(f"Accuracy: {accuracies[i]}\n")

# Вычисление средней точности
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy: {average_accuracy}")

    No System   mount       price         con        snow         ice  \
0    1     SK       F  (149, 149)  (1.9, 1.9)  (2.5, 2.5)  (4.0, 4.0)   
1    2    SRK  F or R  (520, 520)  (2.1, 2.1)  (0.8, 0.8)  (3.8, 3.8)   
2    3    SRK  F or R  (389, 389)  (2.0, 2.0)  (2.2, 2.2)  (3.3, 3.3)   
3    4     SK       F  (213, 213)  (1.7, 1.7)  (2.0, 2.0)  (2.4, 2.4)   
4    5    SMS  F or R  (598, 598)  (1.6, 1.6)  (2.4, 2.4)  (7.0, 7.0)   
5    6     SK       F  (109, 109)  (2.0, 2.0)  (1.9, 1.9)  (2.4, 2.4)   
6    7    SRK  F or R  (325, 325)  (2.0, 2.0)  (2.1, 2.1)  (3.2, 3.2)   
7    8    SMS  F or R  (498, 498)  (1.5, 1.5)  (3.3, 3.3)  (3.5, 3.5)   
8    9    SRK  F or R  (396, 396)  (2.8, 2.8)  (2.1, 2.1)  (3.1, 3.1)   
9   10     SK       F  (160, 160)  (1.7, 1.7)  (1.9, 1.9)  (1.6, 1.6)   
10  11    SRK  F or R  (389, 389)  (2.0, 2.0)  (2.2, 2.2)  (3.3, 3.3)   
11  12    SRK       F  (298, 298)  (2.5, 2.5)  (2.3, 2.3)  (3.3, 3.3)   
12  13     SK       F  (206, 206)  (1.9, 1.9)  (1.4

# Task 1.4

In [18]:
import pandas as pd
data = {
    "No": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
    "System": ["SK", "SRK", "SRK", "SK", "SMS", "SK", "SRK", "SMS", "SRK", "SK", "SRK", "SRK", "SK", "SMS"],
    "mount": ["F", "F or R", "F or R", "F", "F or R", "F", "F or R", "F or R", "F or R", "F", "F or R", "F", "F", "F or R"],
    "price": [149, 520, 389, 213, 598, 109, 325, 498, 396, 160, 389, 298, 206, 684],
    "con": [1.9, 2.1, 2.0, 1.7, 1.6, 2.0, 2.0, 1.5, 2.8, 1.7, 2.0, 2.5, 1.9, 1.7],
    "snow": [2.5, 0.8, 2.2, 2.0, 2.4, 1.9, 2.1, 3.3, 2.1, 1.9, 2.2, 2.3, 1.4, 3.3],
    "ice": [4.0, 3.8, 3.3, 2.4, 7.0, 2.4, 3.2, 3.5, 3.1, 1.6, 3.3, 3.3, 1.8, 4.4],
    "dur": [3.8, 2.3, 4.3, 3.4, 2.8, 3.7, 2.8, 2.0, 2.5, 3.7, 4.3, 2.8, 2.7, 2.2],
    "Accegrade": ["T", "F", "T", "F", "F", "F", "F", "T", "T", "F", "T", "T", "F", "T",]
}

test_data = {
    "No": [15, 16, 17],
    "System": ["SK", "SK", "SK"],
    "mount": ["F", "F", "F"],
    "price": [99, 140, 215],
    "con": [2.8, 2.6, 2.3],
    "snow": [2.2, 2.3, 3.8],
    "ice": [2.5, 3.3, 4.8],
    "dur": [4.0, 3.4, 2.3],
    "Accegrade": ["T", "T", "T"]
}

df = pd.DataFrame(data)
test_df = pd.DataFrame(test_data)
combined_data = pd.concat([df, test_df]).reset_index(drop=True)

train_data = combined_data[combined_data['No'] != 15]
example_15 = combined_data[combined_data['No'] == 15].to_dict(orient='records')[0]
trainX = train_data.copy()
for col in ["price", "con", "snow", "ice", "dur"]:
    trainX[col] = train_data[col].apply(lambda x: (x, x))
banks = trainX.to_dict(orient='records')

# Преобразование атрибутов примера 15 в новые интервалы
for col in ["price", "con", "snow", "ice", "dur"]:
    example_15[col] = (example_15[col], 999)

# Функция для вычисления пересечения интервалов
def interval_intersection(interval1, interval2):
    return (min(interval1[0], interval2[0]), 999)

# Функция для вычисления IntSec
def compute_intsec(bank, test_bank):
    intsec = {}
    for key in bank:
        if key in test_bank and key != 'No' and key != 'Accegrade':  # Исключаем ненужные ключи
            if isinstance(bank[key], tuple):  # Для интервальных атрибутов
                intsec[key] = interval_intersection(bank[key], test_bank[key])
            elif bank[key] == test_bank[key]:  # Для номинальных атрибутов
                intsec[key] = bank[key]
    return intsec

# Функция для вычисления Ext
def compute_ext(banks, intsec):
    ext = []
    for index, bank in enumerate(banks):
        match = True
        for key in intsec:
            if key in bank:
                if isinstance(intsec[key], tuple):  # Для интервальных атрибутов
                    if not (intsec[key][0] <= bank[key][0] <= intsec[key][1]):
                        match = False
                        break
                else:  # Для номинальных атрибутов
                    if intsec[key] != bank[key]:
                        match = False
                        break
            else:
                match = False
                break
        if match:
            ext.append(index)
    return ext

# Алгоритм Ленивой Классификации
def lazy_classification(banks, test_bank):
    for bank in banks:
        intsec = compute_intsec(bank, test_bank)
        #print(f"{bank['No']}:IntSec -  {intsec}")
        ext = compute_ext(banks, intsec)
        #print(f"{bank['No']}:Ext - {ext}")

        if len(ext) > 0:  # Проверка, есть ли элементы в Ext
            if all(banks[i]["Accegrade"] == "T" for i in ext):
                return "T"
            elif all(banks[i]["Accegrade"] == "F" for i in ext):
                return "F" 
            
classification_result = lazy_classification(banks, example_15)
print(f"Classification for example 15: {classification_result}")
len(banks)

Classification for example 15: T


16

In [19]:
from sklearn.model_selection import KFold

# Объединение исходного и тестового наборов данных
combined_data = pd.concat([df, test_df]).reset_index(drop=True)
print(combined_data)
# Подготовка к 5-кратной перекрёстной проверке
kf = KFold(n_splits=5, shuffle=True, random_state=42)
def run_lazy_classification(train, test):
    banks = train.to_dict(orient='records')
    test_banks = test.to_dict(orient='records')

    correct_predictions = 0
    for test_bank in test_banks:
        predicted_label = lazy_classification(banks, test_bank)  # Предсказанное значение
        true_label = test_bank['Accegrade']  # Истинное значение

        if predicted_label == true_label:
            correct_predictions += 1

    accuracy = correct_predictions / len(test_banks)
    return accuracy

# Инициализация списка для хранения результатов точности
accuracies = []

custom_test_indices = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8],
    [9, 10, 11],
    [12, 13, 14]
]

for test_index in custom_test_indices:
    # Индексы для тренировочного набора - все, кроме тестовых
    train_index = [i for i in range(len(combined_data)) if i not in test_index]
    
    train, test = combined_data.iloc[train_index], combined_data.iloc[test_index]
    accuracy = run_lazy_classification(train, test)
    accuracies.append(accuracy)

# Просмотр результатов
for i, test_index in enumerate(custom_test_indices):
    print(f"Fold {i+1}:")
    print(f"Train Indices: {[i for i in range(len(combined_data)) if i not in test_index]}")
    print(f"Test Indices: {test_index}")
    print(f"Accuracy: {accuracies[i]}\n")

# Вычисление средней точности
average_accuracy = sum(accuracies) / len(accuracies)
print(f"Average Accuracy: {average_accuracy}")

    No System   mount  price  con  snow  ice  dur Accegrade
0    1     SK       F    149  1.9   2.5  4.0  3.8         T
1    2    SRK  F or R    520  2.1   0.8  3.8  2.3         F
2    3    SRK  F or R    389  2.0   2.2  3.3  4.3         T
3    4     SK       F    213  1.7   2.0  2.4  3.4         F
4    5    SMS  F or R    598  1.6   2.4  7.0  2.8         F
5    6     SK       F    109  2.0   1.9  2.4  3.7         F
6    7    SRK  F or R    325  2.0   2.1  3.2  2.8         F
7    8    SMS  F or R    498  1.5   3.3  3.5  2.0         T
8    9    SRK  F or R    396  2.8   2.1  3.1  2.5         T
9   10     SK       F    160  1.7   1.9  1.6  3.7         F
10  11    SRK  F or R    389  2.0   2.2  3.3  4.3         T
11  12    SRK       F    298  2.5   2.3  3.3  2.8         T
12  13     SK       F    206  1.9   1.4  1.8  2.7         F
13  14    SMS  F or R    684  1.7   3.3  4.4  2.2         T
14  15     SK       F     99  2.8   2.2  2.5  4.0         T
15  16     SK       F    140  2.6   2.3 